Домашняч работа 5.
Ермаченко Борис



1

In [1]:
from PIL import Image
import PIL.Image

from urllib.request import urlopen
import io
import PIL.ExifTags
import numpy as np
import cv2


In [11]:
img = PIL.Image.open('GOPR01170000.jpg')
exif_data = img._getexif()


In [12]:
exif = {
    PIL.ExifTags.TAGS[k]: v
    for k, v in img._getexif().items()
    if k in PIL.ExifTags.TAGS
}

In [13]:
print('exif ', exif)

exif  {'ColorSpace': 1, 'ExifImageWidth': 960, 'ExifImageHeight': 540, 'ResolutionUnit': 2, 'ExifOffset': 172, 'Software': 'Adobe Photoshop CC 2019 (Macintosh)', 'Orientation': 1, 'DateTime': '2019:01:20 17:55:55', 'XResolution': (720000, 10000), 'YResolution': (720000, 10000)}


In [20]:
print('XResolution ', exif['XResolution'][0])

XResolution  720000


In [15]:
#  Найдем координаты центра
Cx = exif['ExifImageWidth'] / 2
Cy = exif['ExifImageHeight'] / 2
print('Cx =', Cx, 'Cy=', Cy)

Cx = 480.0 Cy= 270.0


In [22]:
# Найдем фокусные расстояния
mmInch = 25.4
F_length = 24
Fx = F_length * exif['XResolution'][0] / mmInch
Fy = F_length * exif['YResolution'][0] / mmInch
print('Fx =', Fx, 'Fy=', Fy)

Fx = 680314.9606299213 Fy= 680314.9606299213


In [25]:
    K = np.array([[Fx, 0, Cx],
                  [0, Fy, Cy],
                  [0, 0, 1]])
    print("K= ", K)

K=  [[6.80314961e+05 0.00000000e+00 4.80000000e+02]
 [0.00000000e+00 6.80314961e+05 2.70000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


2

In [2]:
cv_file = cv2.FileStorage("camera.xml", cv2.FILE_STORAGE_READ)


In [4]:
matrix = cv_file.getNode("camera_matrix").mat()
print("read matrix\n", matrix)

read matrix
 [[425.81151398   0.         481.75802556]
 [  0.         435.33255746 267.43704186]
 [  0.           0.           1.        ]]


In [6]:
dist_coeffs = cv_file.getNode('distortion_coefficients').mat()
print("dist_coeffs\n", dist_coeffs)

dist_coeffs
 [[ 4.44290018e+00]
 [ 2.76490305e+00]
 [-4.52039013e-03]
 [-1.82178053e-03]
 [ 0.00000000e+00]
 [ 4.70565765e+00]
 [ 3.93432913e+00]
 [ 3.39107454e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


In [11]:
img = cv2.imread('GOPR01170000.jpg')


In [14]:

h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(matrix,dist_coeffs,(w,h),1,(w,h))


In [17]:
# undistort
dst = cv2.undistort(img, matrix, dist_coeffs, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('result.png',dst)

True